# Data Science Master Class

## Predykcja cen mieszkań w Moskwie

Inicjatywa organizowana przez DataWorkshop 


## Importujemy biblioteki 

In [ ]:
import pandas as pd
import numpy as np
import helper as h

import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error

### Wczytujemy i łączmy dane

In [ ]:
df_train = pd.read_hdf("../input/train_data.h5")
df_train['price'] = df_train['price'].map(h.parse_price)

df_test = pd.read_hdf("../input/test_data.h5")

df = pd.concat([df_train, df_test])
print(df.shape, df.shape)

(34180, 8) (34180, 8)


### Params

In [ ]:
params = df["params"].apply(pd.Series)
params = params.fillna(-1)

if "Охрана:" not in df:
    df = pd.concat([df, params], axis=1)
    
    obj_feats = params.select_dtypes(object).columns

    for feat in obj_feats:
        df["{}_cat".format(feat)] = df[feat].factorize()[0]

cat_feats = [x for x in df.columns if "_cat" in x]
cat_feats

['Охрана:_cat',
 'Тип здания:_cat',
 'Тип объекта:_cat',
 'Количество корпусов:_cat',
 'Тип объявления:_cat',
 'Застройщик:_cat',
 'Общая площадь:_cat',
 'Дата публикации:_cat',
 'Количество комнат:_cat',
 'Парковка:_cat',
 'Дата  обновления:_cat',
 'Количество этажей:_cat',
 'Сдача:_cat',
 'Комиссия агенту:_cat',
 'Высота потолков:_cat',
 'Этаж:_cat',
 'Этап строительства:_cat',
 'Новостройка:_cat',
 'Количество квартир:_cat',
 'Класс жилья:_cat',
 'Адрес:_cat',
 'Лифт:_cat',
 'Вид из окна:_cat',
 'Возможна ипотека:_cat',
 'Год постройки:_cat',
 'Этажность:_cat',
 'Мусоропровод:_cat',
 'Ремонт:_cat',
 'Площадь кухни:_cat',
 'Жилая комната:_cat',
 'Тип санузла:_cat',
 'Тип балкона:_cat',
 'Мебель на кухне:_cat',
 'Холодильник:_cat',
 'Интернет:_cat',
 'Мебель:_cat',
 'Телефон:_cat',
 'Свободная планировка:_cat',
 'Управляющая компания:_cat',
 'Количество подъездов:_cat',
 'Тип дома:_cat',
 'Тип комнат:_cat',
 'Покрытие пола:_cat',
 'Серия:_cat',
 'Тип перекрытия:_cat',
 'Возможен торг:

### Powierzchnia całkowita



In [ ]:
def parse_area(val):
    if isinstance(val, int): return val
    if isinstance(val, float): return val
    
    return float(val.split("м")[0].replace(" ", ""))

df["area"] = df["Общая площадь:"].map(h.parse_area)

Powierzchnia kuchni

In [ ]:
df["kitchen_area"] = df["Площадь кухни:"].map(parse_area)

## Sprawdzanie

In [ ]:
def check_model(df, feats, model, cv=5, scoring="neg_mean_absolute_error"):
    df_train = df[ ~df["price"].isnull() ].copy()
    df_test = df[ df["price"].isnull() ].copy()

    X_train = df_train[feats]
    y_train = df_train["price"]
    
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring=scoring)
    return np.mean(scores), np.std(scores)

In [ ]:
feats = ["area", "kitchen_area"] + cat_feats

check_model(df, feats, xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0))

(-3.818916642505762, 0.2181521283268608)


### Check Log Model



In [ ]:
def check_log_model(df, feats, model, cv=5, scoring="neg_mean_absolute_error"):
    df_train = df[ ~df["price"].isnull() ].copy()

    X = df_train[feats]
    y = df_train["price"]
    y_log = np.log(y)
    
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    scores = []
    for train_idx, test_idx in cv.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_log_train, y_test = y_log.iloc[train_idx], y.iloc[test_idx]

        model = xgb.XGBRegressor(max_depth=5, n_estimators=50, random_state=0)
        model.fit(X_train, y_log_train)
        y_log_pred = model.predict(X_test)
        y_pred = np.exp(y_log_pred)

        score = mean_absolute_error(y_test, y_pred)
        scores.append(score)

    return np.mean(scores), np.std(scores)

In [ ]:
feats = ["area", "kitchen_area"] + cat_feats

check_log_model(df, feats, xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0))

(3.419673246298367, 0.3060063523202168)

In [ ]:
df['balcony_type'] = df['Тип балкона:'].map(lambda x: str(x) if x == -1 else x).factorize()[0]
df['balcony_type'].value_counts()

0    25779
3     4520
1     3611
2      125
4       89
5       56
Name: balcony_type, dtype: int64

In [ ]:
df["Класс жилья:"].value_counts()

Комфорт класс    13895
Бизнес класс      9937
-1                7416
Элит класс        2057
Эконом класс       875
Name: Класс жилья:, dtype: int64

In [ ]:
df["housing_class"] = df["Класс жилья:"].map(lambda x: str(x) if x == -1 else x).factorize()[0]


In [ ]:
df["Этаж:"].value_counts()

2        658
3        485
4        446
6        419
8        403
        ... 
22/82      1
33/38      1
52/70      1
14/42      1
69/73      1
Name: Этаж:, Length: 1241, dtype: int64

In [ ]:
def parse_floor(val):
    if isinstance(val, int): return val
    if isinstance(val, str):
        return val.split('/')[0]
    return val

In [ ]:
df["floor"] = df["Этаж:"].map(parse_floor).map(lambda x: int(x))
df["floor"]

0         8
1         6
2        20
3        27
4         2
         ..
11443     4
11444    11
11445    21
11446     4
11447    17
Name: floor, Length: 34180, dtype: int64

In [ ]:
df["Высота потолков:"].value_counts()

-1        7486
3 м       5773
2 м       4002
2.8 м     2782
2.7 м     2733
          ... 
2.45 м       1
7 м          1
2.67 м       1
3.03 м       1
0.05 м       1
Name: Высота потолков:, Length: 68, dtype: int64

In [ ]:
def parse_ceiling_height(val):
    if isinstance(val, float): return val
    if isinstance(val, str):
        return float(val.split("м")[0].replace(" ", ""))
    return val


In [ ]:
df["ceiling_height"] = df["Высота потолков:"].map(parse_ceiling_height).map(lambda x: float(x))


In [ ]:
df["ceiling_height"].value_counts(normalize=True)

-1.00    0.219017
 3.00    0.168900
 2.00    0.117086
 2.80    0.081393
 2.70    0.079959
           ...   
 4.70    0.000029
 3.03    0.000029
 6.30    0.000029
 2.45    0.000029
 2.82    0.000029
Name: ceiling_height, Length: 68, dtype: float64

In [ ]:
df["Ремонт:"].value_counts()

-1               29612
евро              4241
дизайнерский       119
косметический       99
без ремонта         41
без отделки         37
с отделкой          31
Name: Ремонт:, dtype: int64

In [ ]:
df["renovation"] = pd.factorize(df["Ремонт:"])[0]

In [ ]:
df["renovation"].value_counts()

0    29612
1     4241
2      119
4       99
5       41
3       37
6       31
Name: renovation, dtype: int64

In [ ]:
df["Возможна ипотека:"].value_counts()

-1    19926
да    14254
Name: Возможна ипотека:, dtype: int64

In [ ]:
df["mortgage"] = df["Возможна ипотека:"].apply(lambda x: 1 if x == "да" else 0)
df["mortgage"]

0        0
1        1
2        0
3        1
4        0
        ..
11443    0
11444    1
11445    1
11446    0
11447    1
Name: mortgage, Length: 34180, dtype: int64

In [ ]:
df["Комиссия агенту:"].value_counts()

без комиссии    33929
-1                245
1%                  2
10%                 2
3%                  1
2%                  1
Name: Комиссия агенту:, dtype: int64

In [ ]:
df["no_commission"] = df["Комиссия агенту:"].apply(lambda x: 1 if x == "без комиссии" else 0) #pozostałe wartości pomijam


In [ ]:
df["Лифт:"].value_counts()

да    20033
-1    14147
Name: Лифт:, dtype: int64

In [ ]:
df["Elevator"] = df["Лифт:"].apply(lambda x: 1 if x == "да" else 0)
df["Elevator"]

0        0
1        1
2        1
3        1
4        0
        ..
11443    1
11444    1
11445    1
11446    1
11447    1
Name: Elevator, Length: 34180, dtype: int64

In [ ]:
df["geo_block"]

0             [г. Москва, Лианозово, г. Москва, Лианозово]
1                 [г. Москва, Ховрино, г. Москва, Ховрино]
2        [г. Москва, ул Лобачевского, г. Москва, ул Лоб...
3        [г. Москва, Ховрино, ул Дыбенко, г. Москва, Хо...
4         [г. Москва, Даниловский, г. Москва, Даниловский]
                               ...                        
11443                               [г. Москва, г. Москва]
11444    [г. Москва, Хорошёво-Мнёвники, г. Москва, Хоро...
11445             [г. Москва, Ховрино, г. Москва, Ховрино]
11446    [Новая Москва, п. Внуковское, г. Москва, Новая...
11447    [г. Москва, Ярославский, ул Красная Сосна, г. ...
Name: geo_block, Length: 34180, dtype: object

In [ ]:
df["geo_block"].map(lambda x: x[0])

0           г. Москва
1           г. Москва
2           г. Москва
3           г. Москва
4           г. Москва
             ...     
11443       г. Москва
11444       г. Москва
11445       г. Москва
11446    Новая Москва
11447       г. Москва
Name: geo_block, Length: 34180, dtype: object

In [ ]:
ids, labels = df["geo_block"].map(lambda x: x[0]).factorize()
ids, labels

(array([0, 0, 0, ..., 0, 1, 0]),
 Index(['г. Москва', 'Новая Москва', 'г. Зеленоград', 'п. Свиблово',
        'п. Некрасовка', 'п. Рублево', 'п. Новобратцевский', 'д. Сколково',
        'п. Подушкино', 'п. Крюково', 'п. Главмосстроя', 'пгт Акулово',
        'п. Алабушево'],
       dtype='object'))

In [ ]:
df["geo_block_0_cat"] = df["geo_block"].map(lambda x: x[0]).factorize()[0]
df["geo_block_0_cat"]

0        0
1        0
2        0
3        0
4        0
        ..
11443    0
11444    0
11445    0
11446    1
11447    0
Name: geo_block_0_cat, Length: 34180, dtype: int64

In [ ]:
df["Вид из окна:"].value_counts()

-1          23019
на улицу     5633
во двор      5528
Name: Вид из окна:, dtype: int64

In [ ]:
df["view_from_window"] = df["Вид из окна:"].factorize()[0]
df["view_from_window"]

0        0
1        1
2        0
3        1
4        0
        ..
11443    0
11444    0
11445    1
11446    0
11447    2
Name: view_from_window, Length: 34180, dtype: int64

In [ ]:
feats = ["area", "housing_class", "balcony_type", "kitchen_area", "floor", "renovation", "mortgage", "no_commission", "Elevator", "geo_block_0_cat", "view_from_window"] + cat_feats

check_log_model(df, feats, xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0))

(3.248558603503427, 0.29448240024794914)

In [ ]:
# z "ceiling_height" wynik był gorszy, więc cecha ta nie została uwzględniona w końcowym modelu
feats = ["area", "housing_class", "balcony_type", "kitchen_area", "floor", "ceiling_height", "renovation", "mortgage", "no_commission", "Elevator", "geo_block_0_cat", "view_from_window"] + cat_feats

check_log_model(df, feats, xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0))

(3.2498277115047047, 0.3216307590001552)

## Kaggle submit


In [ ]:
feats = ["area", "housing_class", "balcony_type", "kitchen_area", "floor", "renovation", "mortgage", "no_commission", "Elevator", "geo_block_0_cat", "view_from_window"] + cat_feats
df_train = df[ ~df["price"].isnull() ].copy()
df_test = df[ df["price"].isnull() ].copy()

X_train = df_train[feats]
y_train = df_train["price"]
y_log_train = np.log(y_train)

X_test = df_test[feats]

model = xgb.XGBRegressor(max_depth=5, n_estimators=50, learning_rate=0.3, random_state=0)
model.fit(X_train, y_log_train)
y_log_pred = model.predict(X_test)
y_pred = np.exp(y_log_pred)


df_test["price"] = y_pred
df_test[ ["id", "price"] ].to_csv("../output/xgb_log_area.csv", index=False)

Kaggle result: 2.95935